In [1]:
import pandas as pd
import os
tama_df = pd.read_csv('tama.csv', encoding='cp1255')
tama_df.columns

Index(['MezheYeshut', 'GISId', 'MisparYeshut', 'MisparYeshutAv',
       'AchuzHachtamot', 'AchuzHachtamotMitcham', 'Bayt', 'GushChelka',
       'KamutDirot', 'KamutTziburi', 'kaymetminhelet', 'KodMinhelet',
       'KtovetAishKesher', 'MachozMinhal', 'MahutBakasha', 'MechirMemutza',
       'mitve', 'Rechov', 'SemelIishuv', 'ShemMitcham', 'ShnatBakasha',
       'ShnatHryter', 'ShnatIchlus', 'Status', 'TaarichMechiraRishona',
       'YachadKayam', 'YachadMutza', 'YachadTabo', 'YachadTosafti', 'Yazam',
       'Yeshut', 'Yeshuv', 'BakashaLeHeter', 'Heter1', 'Ichlus1'],
      dtype='object')

### restructure tama data - create new tama data with new columns based on the original tama data

In [2]:
years_list, cities_list = list(range(2005, 2023)), list(set(tama_df['Yeshuv']))
years_num, cities_num = len(years_list), len(cities_list)
list_of_cities_list = [[city] * years_num for city in cities_list]
cities = []
for city in list_of_cities_list:
    cities.extend(city)
restructure_tama_df = pd.DataFrame(data={'year': years_list * cities_num, 
                                                       'Yeshuv': cities
                                                      })
restructure_tama_df.sort_values(by=['Yeshuv', 'year'], ignore_index=True, inplace=True)
restructure_tama_df

,year,Yeshuv
0,2005,אור יהודה
1,2006,אור יהודה
2,2007,אור יהודה
3,2008,אור יהודה
4,2009,אור יהודה
...,...,...
1075,2018,תל אביב יפו
1076,2019,תל אביב יפו
1077,2020,תל אביב יפו
1078,2021,תל אביב יפו


In [3]:
semel_yeshuv_map = tama_df[['SemelIishuv', 'Yeshuv']].drop_duplicates().sort_values(by=['SemelIishuv', 'Yeshuv'])

In [4]:
restructure_tama_df = restructure_tama_df.merge(
    semel_yeshuv_map, on=['Yeshuv']).sort_values(by=['Yeshuv', 'year'], ignore_index=True)
restructure_tama_df

,year,Yeshuv,SemelIishuv
0,2005,אור יהודה,2400
1,2006,אור יהודה,2400
2,2007,אור יהודה,2400
3,2008,אור יהודה,2400
4,2009,אור יהודה,2400
...,...,...,...
1075,2018,תל אביב יפו,5000
1076,2019,תל אביב יפו,5000
1077,2020,תל אביב יפו,5000
1078,2021,תל אביב יפו,5000


In [5]:
total_requests_per_city_and_year_df = tama_df.groupby(by=['SemelIishuv', 'Yeshuv', 'ShnatBakasha'], as_index=False).size()
total_requests_per_city_and_year_df.rename(
    columns={'ShnatBakasha': 'year', 'size': 'total_requests'}, inplace=True)


total_approved_requests_per_city_and_year_df = tama_df.groupby(by=['SemelIishuv', 'Yeshuv', 'ShnatHryter'], as_index=False).size()
total_approved_requests_per_city_and_year_df.rename(
    columns={'ShnatHryter': 'year', 'size': 'total_approved_requests'}, inplace=True)

restructure_tama_df = restructure_tama_df.merge(
    total_requests_per_city_and_year_df, on=['SemelIishuv', 'Yeshuv', 'year'], how='left').merge(
    total_approved_requests_per_city_and_year_df, on=['SemelIishuv', 'Yeshuv', 'year'], how='left')

restructure_tama_df = restructure_tama_df.fillna(0)
restructure_tama_df.sort_values(by=['Yeshuv', 'year'], ignore_index=True, inplace=True)
restructure_tama_df.rename(columns={'Yeshuv': 'hebrew_city_name', 'SemelIishuv': 'city_code'}, inplace=True)
restructure_tama_df

,year,hebrew_city_name,city_code,total_requests,total_approved_requests
0,2005,אור יהודה,2400,0.0,0.0
1,2006,אור יהודה,2400,0.0,0.0
2,2007,אור יהודה,2400,0.0,0.0
3,2008,אור יהודה,2400,1.0,0.0
4,2009,אור יהודה,2400,0.0,0.0
...,...,...,...,...,...
1075,2018,תל אביב יפו,5000,164.0,83.0
1076,2019,תל אביב יפו,5000,116.0,149.0
1077,2020,תל אביב יפו,5000,238.0,115.0
1078,2021,תל אביב יפו,5000,240.0,124.0


In [6]:
cols = ['hebrew_city_name', 'city_code', 'year', 'total_requests', 'total_approved_requests']
restructure_tama_df = restructure_tama_df[cols]
restructure_tama_df.sort_values(by=cols, ignore_index=True, inplace=True)
restructure_tama_df.to_csv('processed_tama_data.csv', index=False)
restructure_tama_df

,hebrew_city_name,city_code,year,total_requests,total_approved_requests
0,אור יהודה,2400,2005,0.0,0.0
1,אור יהודה,2400,2006,0.0,0.0
2,אור יהודה,2400,2007,0.0,0.0
3,אור יהודה,2400,2008,1.0,0.0
4,אור יהודה,2400,2009,0.0,0.0
...,...,...,...,...,...
1075,תל אביב יפו,5000,2018,164.0,83.0
1076,תל אביב יפו,5000,2019,116.0,149.0
1077,תל אביב יפו,5000,2020,238.0,115.0
1078,תל אביב יפו,5000,2021,240.0,124.0
